In [1]:
import requests
import re
from tqdm import tqdm
import time

In [2]:
basepath = 'http://pogovorki.net/cat{:02}/p{}.html'
html_re = '''<p class="text" align="justify">(.+?)\n?<a class="pogovorka".*?</p>'''
paths = [basepath.format(cat, p) for cat in range(1,27) for p in range(1,57)]

In [3]:
%%time
proverbs = []
for path in tqdm(paths):
    try:
        r = requests.get(path)
        if r.status_code == 200:
            provb = re.findall(html_re, r.text, re.DOTALL)
            proverbs += provb
        else: 
            print(path, r.status_code)
    except Exception as e:
        print(path, e)

100%|██████████████████████████████████████████████████████████████████████████████| 1456/1456 [11:22<00:00,  2.13it/s]


Wall time: 11min 22s


In [65]:
len(proverbs)

5746

In [73]:
%%time
for i, p in enumerate(proverbs):
    replacements = [
        ('\s?Ср\.\s?', ' '),
        ('\(.*?\)', ''),
        ('(\S) +(\.)', '\g<1>\g<2>'),
        ('[a-zA-Z]*', ''),
        ('^\W+(\w+)', '\g<1>'),
        (' +', ' '),
    ]
    for old, new in replacements:
        p = re.sub(old, new, p)
    proverbs[i] = p

Wall time: 211 ms


In [74]:
with open('proverbs.txt', 'w', encoding='utf-8') as f:
    for i in proverbs:
        f.write("{}\n".format(i))